In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/ML Project/Project Report/"


In [ ]:
with open(path+'Models/ohe_model.pkl', 'rb') as f:
    vectorizer, loaded_model = pickle.load(f)

In [ ]:
def get_new_df(df):
  new_df = pd.DataFrame(columns=['ID', 'positive', 'negative'])
  curr = df.iloc[0]['ID']
  positive = 0
  negative = 0
  if df.iloc[0]['Predictions'] == "negative":
    negative = negative + 1
  elif df.iloc[0]['Predictions'] == "positive":
    positive = positive + 1
  for i in range(1, len(df)):
    if curr != df.iloc[i]['ID']:
      df2 = {'ID': curr, 'positive':  positive/(positive + negative ), 'negative': negative/(positive + negative )}
      new_df = new_df.append(df2, ignore_index = True)
      # new_df.insert(curr, positive/(positive + negative ), negative/(positive + negative ))
      # new_df["ID"] = curr
      # new_df["positive"] = positive/(positive + negative )
      # new_df["negative"] = negative/(positive + negative )
      curr = df.iloc[i]['ID']
      positive = 0
      negative = 0

    if df.iloc[i]['Predictions'] == "negative":
      negative = negative + 1
    elif df.iloc[i]['Predictions'] == "positive":
      positive = positive + 1

  df2 = {'ID': curr, 'positive':  positive/(positive + negative ), 'negative': negative/(positive + negative )}
  new_df = new_df.append(df2, ignore_index = True) 
  return new_df


In [ ]:
final_df = pd.DataFrame(columns=['ID', 'positive', 'negative'])
for i in range(1, 17):
  movieComments = pd.read_csv(path+"Datasets/movie_comments/movieCommentsCleaned/movieCommentsCleaned"+str(i)+".csv")
  movieComments =  movieComments.dropna()
  x = vectorizer.transform(movieComments['cleaned_comments']).toarray()
  pred = loaded_model.predict(x)
  movieComments["Predictions"] = pred
  df = movieComments[['ID', 'Predictions']].copy()
  part_final_df = get_new_df(df)
  # print(part_final_df)
  final_df = pd.concat([final_df, part_final_df])


<ipython-input-9-3a95da4c5bd8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df2, ignore_index = True)
<ipython-input-9-3a95da4c5bd8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df2, ignore_index = True)
<ipython-input-9-3a95da4c5bd8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df2, ignore_index = True)
<ipython-input-9-3a95da4c5bd8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df2, ignore_index = True)
<ipython-input-9-3a95da4c5bd8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

**Saving the dataset**

In [ ]:
# final_df.to_csv("oneHotEncodedValues.csv")

In [ ]:
pos_neg_df = pd.read_csv(path+"Datasets/oneHotEncodedValues.csv")
movies_df = pd.read_csv(path+"Datasets/movie_success_rate.csv")
movies_df = movies_df[['Title', 'Rating']].copy()
id_df = pd.read_csv(path+"Datasets/movies_ids_updated.csv")

In [ ]:
result = pd.merge(id_df, movies_df, on=["Title"])
final_result = pd.merge(result, pos_neg_df, on=["ID"])

In [ ]:
final_result = final_result[['Title', 'ID', 'positive', 'negative', 'Rating']].copy()

l1 = []
for i in range(len(final_result)):
  if(final_result.iloc[i]['Rating'] > 7):
    l1.append('Good')
  elif(final_result.iloc[i]['Rating'] > 5):
    l1.append('Average')
  elif(final_result.iloc[i]['Rating'] > 0):
    l1.append('Bad')

final_result['result_category'] = l1

In [ ]:
final_result

,Title,ID,positive,negative,Rating,result_category
0,Guardians of the Galaxy,d96cjJhvlMA,0.744898,0.255102,8.1,Good
1,Prometheus,HHcHYisZFLU,0.585859,0.414141,7.0,Average
2,Split,KIpGKumxiGg,0.602041,0.397959,7.3,Good
3,Suicide Squad,CmRih_VtVAs,0.762887,0.237113,6.2,Average
4,The Great Wall,avF6GHyyk5c,0.555556,0.444444,6.1,Average
...,...,...,...,...,...,...
637,Life as We Know It,mqzjDrrZIdE,0.580000,0.420000,6.6,Average
638,28 Weeks Later,ljqY7qrnycw,0.547619,0.452381,7.0,Average
639,Warm Bodies,_TDYhTNpUzo,0.738095,0.261905,6.9,Average
640,Blue Jasmine,KK7_xLxf4o0,0.764706,0.235294,7.3,Good


In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import keras

In [ ]:
data = final_result[['positive', 'negative', 'result_category']].values

In [ ]:
X = data[:,0:2]
X = np.asarray(X).astype('float32')
Y = data[:,2]

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)


In [ ]:
y = np_utils.to_categorical(encoded_Y)

In [ ]:
y

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
Y_train

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential([
    Dense(16, activation='relu'),
    Dense(32, activation='sigmoid'),
    Dense(3, activation='sigmoid'),
])



In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, Y_train, epochs=300, batch_size=50)

Epoch 1/300
11/11 [==============================] - 1s 3ms/step - loss: 1.0129 - accuracy: 0.5602
Epoch 2/300
11/11 [==============================] - 0s 3ms/step - loss: 0.9299 - accuracy: 0.5602
Epoch 3/300
11/11 [==============================] - 0s 3ms/step - loss: 0.8768 - accuracy: 0.5602
Epoch 4/300
11/11 [==============================] - 0s 2ms/step - loss: 0.8444 - accuracy: 0.5602
Epoch 5/300
11/11 [==============================] - 0s 2ms/step - loss: 0.8237 - accuracy: 0.5602
Epoch 6/300
11/11 [==============================] - 0s 3ms/step - loss: 0.8100 - accuracy: 0.5602
Epoch 7/300
11/11 [==============================] - 0s 3ms/step - loss: 0.8019 - accuracy: 0.5602
Epoch 8/300
11/11 [==============================] - 0s 2ms/step - loss: 0.7967 - accuracy: 0.5602
Epoch 9/300
11/11 [==============================] - 0s 3ms/step - loss: 0.7919 - accuracy: 0.5602
Epoch 10/300
11/11 [==============================] - 0s 3ms/step - loss: 0.7891 - accuracy: 0.5602
Epoch 11/

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.7638936042785645
Test accuracy: 0.6009174585342407
